In [1]:
# import autosklearn.regression
import sklearn.model_selection
import sklearn.metrics
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

<h1>X_train,y_train</h1>

In [3]:
df = []
src = r"C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\train\train"
for file in os.listdir(src):
    df.append(pd.read_csv(os.path.join(src,file)))
for i in df:
    print(list(i))
df = pd.concat(df)
df = df.fillna(0)
df[-10:]

['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed', '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน', '23SoilEC', '23SoilpH', '26N', '26P', '26K', '29SoilTension', 'flow1']
['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed', '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน', '23SoilEC', '23SoilpH', '26N', '26P', '26K', '29SoilTension', 'flow1']
['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed', '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน', '23SoilEC', '23SoilpH', '29SoilTension', 'flow1', '26N', '26P', '26K']
['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed', '13WindDirection', '23SoilWC ค.ชื้

,index,time,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,13WindDirection,23SoilWC ค.ชื้นดิน,23SoilTemp.อุณหภูมิดิน,23SoilEC,23SoilpH,26N,26P,26K,29SoilTension,flow1
104,12_105,04:40:16.090733+00,44.1,31.2,0.0,47.64,0.0,0.0,0.5,3.0,38.8,22.8,0.11,5.0,21.0,95.0,87.0,105.0,0.0
105,12_106,04:42:16.132055+00,44.2,31.2,0.0,49.50,0.0,0.0,0.1,1.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,111.0,0.0
106,12_107,04:44:16.107327+00,45.2,30.8,0.0,50.03,0.0,0.0,1.2,7.0,37.4,22.8,0.11,5.0,21.0,95.0,87.0,113.0,0.0
107,12_108,04:46:16.11276+00,45.9,32.0,0.0,50.06,0.0,0.0,0.6,1.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,110.0,0.0
108,12_109,04:48:16.120498+00,41.4,33.1,0.0,50.36,0.0,0.0,0.7,7.0,37.6,22.8,0.11,5.0,21.0,95.0,87.0,108.0,0.0
109,12_110,04:50:16.123489+00,43.0,33.0,0.0,52.30,0.0,0.0,1.0,5.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,109.0,0.0
110,12_111,04:52:18.185763+00,42.5,32.5,0.0,53.76,0.0,0.0,0.8,4.0,37.4,22.8,0.11,5.0,21.0,95.0,87.0,0.0,0.0
111,12_112,04:54:16.141528+00,41.7,33.1,0.0,54.86,0.0,0.0,0.8,4.0,37.4,22.8,0.11,5.1,21.0,95.0,87.0,109.0,0.0
112,12_113,04:56:20.078654+00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,2.0,36.9,22.8,0.11,5.0,21.0,95.0,87.0,104.0,0.0
113,12_114,04:58:16.116687+00,44.8,32.5,0.0,58.81,0.0,0.0,1.2,7.0,37.4,22.8,0.11,5.1,21.0,95.0,87.0,113.0,0.0


In [4]:
df.columns

Index(['index', 'time', '1Air Humidity', '1AirTemperature', '1LightHigh',
       '1LightLow', '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed',
       '13WindDirection', '23SoilWC ค.ชื้นดิน', '23SoilTemp.อุณหภูมิดิน',
       '23SoilEC', '23SoilpH', '26N', '26P', '26K', '29SoilTension', 'flow1'],
      dtype='object')

In [5]:
x_train = df[['1Air Humidity', '1AirTemperature', '1LightHigh',
                '1LightLow', '11Rain Accumulate (mm)', '11Rain count','12WindSpeed',
                '23SoilEC', '23SoilpH', '23SoilTemp.อุณหภูมิดิน',  '26N', '26P', '26K',
                '29SoilTension', 'flow1']]
y_train = df[['23SoilWC ค.ชื้นดิน']]

In [6]:
x_train.head()

,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,23SoilEC,23SoilpH,23SoilTemp.อุณหภูมิดิน,26N,26P,26K,29SoilTension,flow1
0,97.3,21.8,0.0,9.57,0.0,0.0,0.0,0.11,8.3,23.6,21.0,95.0,87.0,23.0,0.0
1,96.6,22.0,0.0,9.81,0.0,0.0,0.0,0.11,8.3,23.6,21.0,95.0,87.0,22.0,0.0
2,95.9,22.2,0.0,10.22,0.0,0.0,0.0,0.11,8.3,23.6,21.0,95.0,87.0,0.0,0.0
3,95.3,22.4,0.0,10.25,0.0,0.0,0.0,0.11,8.3,23.6,21.0,95.0,87.0,22.0,0.0
4,93.8,22.4,0.0,10.45,0.0,0.0,0.0,0.11,8.3,23.6,21.0,95.0,87.0,23.0,0.0


<h1>X_test,Y_test</h1>

In [7]:
x_test = []
src2 = r'C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\test\test'
for file in os.listdir(src2):
    print(os.path.join(src2,file))
    x_test.append(pd.read_csv(os.path.join(src2,file)))
x_test = pd.concat(x_test)
x_test = x_test.fillna(0)


C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\test\test\set02.csv
C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\test\test\set09.csv


In [8]:
x_test = x_test.drop(['index','time','13WindDirection'],axis=1)            # Drop column
x_test = x_test[x_train.columns]                                           # Reorder
x_test

,1Air Humidity,1AirTemperature,1LightHigh,1LightLow,11Rain Accumulate (mm),11Rain count,12WindSpeed,23SoilEC,23SoilpH,23SoilTemp.อุณหภูมิดิน,26N,26P,26K,29SoilTension,flow1
0,46.2,30.7,65.53,8.79,0.0,0.0,1.3,0.11,7.9,22.4,21.0,95.0,88.0,26.0,0.0
1,46.0,31.4,65.53,8.26,0.0,0.0,0.0,0.11,7.9,22.4,21.0,95.0,88.0,26.0,0.0
2,44.4,32.2,65.53,8.66,0.0,0.0,1.0,0.11,7.9,22.3,21.0,95.0,88.0,0.0,0.0
3,43.0,33.0,65.53,8.68,0.0,0.0,0.0,0.11,7.9,22.4,21.0,95.0,88.0,26.0,0.0
4,42.4,32.9,65.53,8.86,0.0,0.0,0.0,0.11,7.9,22.4,21.0,95.0,88.0,26.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,36.2,35.9,0.00,53.35,0.0,0.0,0.0,0.12,4.9,23.7,27.0,110.0,103.0,48.0,0.0
112,39.8,35.9,0.00,54.13,0.0,0.0,1.9,0.12,4.9,23.7,27.0,110.0,103.0,0.0,0.0
113,38.4,35.8,0.00,54.11,0.0,0.0,0.0,0.12,4.9,23.7,27.0,110.0,103.0,48.0,0.0
114,38.2,35.6,0.00,54.26,0.0,0.0,0.0,0.12,4.9,23.7,27.0,110.0,103.0,45.0,0.0


<hr>
<hr>

<h1>RandomForrest</h1>

In [11]:
rf = RandomForestRegressor()
rf.fit(x_train, y_train)
y_hat = rf.predict(x_test)
y_hat

array([36.703, 37.103, 36.721, 37.395, 37.395, 37.255, 37.209, 37.255,
       34.4  , 34.587, 34.788, 34.551, 34.365, 34.384, 34.378, 35.965,
       37.788, 45.279, 47.179, 48.236, 49.595, 49.176, 49.588, 49.526,
       48.897, 49.56 , 48.966, 49.176, 49.543, 43.225, 49.246, 42.139,
       69.699, 71.363, 69.777, 70.067, 69.828, 70.071, 69.796, 70.264,
       69.458, 68.81 , 67.597, 68.824, 69.123, 67.845, 67.483, 68.927,
       68.516, 66.933, 68.238, 69.468, 68.656, 48.786, 51.02 , 48.117,
       49.064, 50.63 , 49.105, 49.347, 43.382, 47.848, 37.563, 39.573,
       39.598, 38.831, 39.782, 40.18 , 39.825, 40.14 , 39.982, 40.062,
       41.232, 41.41 , 41.605, 41.412, 39.794, 38.821, 40.615, 39.991,
       39.756, 39.441, 39.468, 39.43 , 39.378, 39.375, 38.782, 40.234,
       40.092, 40.155, 40.458, 40.144, 38.226, 40.329, 38.689, 38.669,
       38.624, 40.203, 38.901, 40.323, 39.281, 38.133, 38.102, 38.33 ,
       38.296, 38.232, 38.282, 38.243, 38.299, 38.233, 38.152, 38.18 ,
      

<h1>XGB</h1>

In [13]:
xb = xgb.XGBRegressor()
xb.fit(x_train, y_train)
y_hat = xb.predict(x_test)
y_hat

array([36.17722 , 35.352097, 35.18794 , 35.057014, 34.76025 , 34.802624,
       35.95767 , 35.3987  , 35.335   , 36.084312, 36.30521 , 35.748108,
       36.133545, 36.75612 , 35.51889 , 35.491543, 36.086433, 46.216103,
       50.618473, 49.63104 , 49.10952 , 48.953533, 50.461212, 48.57551 ,
       47.599075, 50.432785, 48.64828 , 50.012016, 50.21013 , 47.260406,
       51.184807, 47.487934, 74.47788 , 75.10791 , 74.227264, 72.53344 ,
       72.42512 , 73.192566, 73.10245 , 74.01222 , 72.78615 , 73.83416 ,
       72.64047 , 73.39079 , 73.76202 , 71.46801 , 71.66819 , 73.82916 ,
       71.449196, 70.48907 , 69.99119 , 72.57456 , 69.9048  , 48.482918,
       50.784348, 48.243828, 48.928043, 48.738495, 48.561687, 46.75708 ,
       43.032818, 46.748627, 37.27694 , 38.16132 , 38.48374 , 36.536892,
       37.91608 , 39.32624 , 39.189075, 39.05152 , 38.71509 , 38.956585,
       42.66302 , 43.110046, 42.83294 , 42.67854 , 39.01226 , 38.907894,
       40.590652, 41.09522 , 38.488308, 38.365738, 

<h1>Feature selection</h1>

In [14]:
xb = xgb.XGBRegressor()
reg_mod = RFECV(xb,
                cv=3,
                step=5
                )

In [15]:
_ = reg_mod.fit(x_train, y_train)

In [26]:
selected_columns = x_train.columns[reg_mod.support_]
print(reg_mod.support_)
print(selected_columns)

[False False False False False False False  True False False False False
 False False False]
Index(['23SoilEC'], dtype='object')


In [24]:
x_train.columns

Index(['1Air Humidity', '1AirTemperature', '1LightHigh', '1LightLow',
       '11Rain Accumulate (mm)', '11Rain count', '12WindSpeed', '23SoilEC',
       '23SoilpH', '23SoilTemp.อุณหภูมิดิน', '26N', '26P', '26K',
       '29SoilTension', 'flow1'],
      dtype='object')

In [29]:
train_data_selected = x_train[selected_columns]
test_data_selected = x_test[selected_columns]

In [31]:
xb = xgb.XGBRegressor()
xb.fit(train_data_selected, y_train)
y_hat = xb.predict(test_data_selected)
y_hat

array([36.166336, 36.166336, 36.166336, 36.166336, 36.166336, 36.166336,
       36.166336, 36.166336, 36.166336, 36.166336, 36.166336, 36.166336,
       36.166336, 36.166336, 36.166336, 36.166336, 36.166336, 44.23318 ,
       44.23318 , 44.23318 , 44.23318 , 44.23318 , 44.23318 , 44.23318 ,
       44.23318 , 44.23318 , 44.23318 , 44.23318 , 44.23318 , 44.23318 ,
       44.23318 , 44.23318 , 62.1559  , 62.1559  , 62.1559  , 62.1559  ,
       62.1559  , 62.1559  , 62.1559  , 62.1559  , 62.1559  , 62.1559  ,
       62.1559  , 62.1559  , 62.1559  , 62.1559  , 62.1559  , 62.1559  ,
       62.1559  , 62.1559  , 62.1559  , 62.1559  , 62.1559  , 44.23318 ,
       44.23318 , 44.23318 , 44.23318 , 44.23318 , 44.23318 , 44.23318 ,
       44.23318 , 44.23318 , 36.166336, 36.166336, 36.166336, 36.166336,
       36.166336, 36.166336, 36.166336, 36.166336, 36.166336, 36.166336,
       36.166336, 36.166336, 36.166336, 36.166336, 36.166336, 36.166336,
       36.166336, 36.166336, 36.166336, 36.166336, 

<hr>

<h1>Model Selection</h1>

In [32]:
param_grid = {
        "colsample_bytree": [0.3, 0.4, 0.5],
        "gamma": [0],
        "learning_rate": [0.4, 0.5, 0.6],
        "max_depth": [1, 2, 3],
        "reg_lambda": [11, 12, 13],
        "scale_pos_weight": [0],
        "subsample": [0.4, 0.5, 0.6],
}

In [33]:
grid_cv = xgb.XGBRegressor()
grid_cv = GridSearchCV(
    grid_cv, 
    param_grid, 
    n_jobs=-1, 
    cv=3, 
    scoring='neg_mean_absolute_error'
)
_ = grid_cv.fit(train_data_selected, y_train)
grid_cv.best_params_

{'colsample_bytree': 0.3,
 'gamma': 0,
 'learning_rate': 0.6,
 'max_depth': 1,
 'reg_lambda': 13,
 'scale_pos_weight': 0,
 'subsample': 0.4}

In [34]:
y_hat = grid_cv.predict(test_data_selected)
y_hat

array([35.946785, 35.946785, 35.946785, 35.946785, 35.946785, 35.946785,
       35.946785, 35.946785, 35.946785, 35.946785, 35.946785, 35.946785,
       35.946785, 35.946785, 35.946785, 35.946785, 35.946785, 44.718544,
       44.718544, 44.718544, 44.718544, 44.718544, 44.718544, 44.718544,
       44.718544, 44.718544, 44.718544, 44.718544, 44.718544, 44.718544,
       44.718544, 44.718544, 63.071476, 63.071476, 63.071476, 63.071476,
       63.071476, 63.071476, 63.071476, 63.071476, 63.071476, 63.071476,
       63.071476, 63.071476, 63.071476, 63.071476, 63.071476, 63.071476,
       63.071476, 63.071476, 63.071476, 63.071476, 63.071476, 44.718544,
       44.718544, 44.718544, 44.718544, 44.718544, 44.718544, 44.718544,
       44.718544, 44.718544, 35.946785, 35.946785, 35.946785, 35.946785,
       35.946785, 35.946785, 35.946785, 35.946785, 35.946785, 35.946785,
       35.946785, 35.946785, 35.946785, 35.946785, 35.946785, 35.946785,
       35.946785, 35.946785, 35.946785, 35.946785, 

<hr>

In [40]:
submission = pd.read_csv(r'C:\Users\Acer\Desktop\Hackathon4\farm-connect-soil-water-content-prediction\sample_submission.csv')

In [41]:
submission

,index,23SoilWC ค.ชื้นดิน
0,02_1,31.1
1,02_2,NaN
2,02_3,NaN
3,02_4,NaN
4,02_5,NaN
...,...,...
224,09_112,NaN
225,09_113,NaN
226,09_114,NaN
227,09_115,NaN


In [42]:
submission['23SoilWC ค.ชื้นดิน'].iloc[1:] = y_hat[1:]

In [43]:
submission

,index,23SoilWC ค.ชื้นดิน
0,02_1,31.100000
1,02_2,35.946785
2,02_3,35.946785
3,02_4,35.946785
4,02_5,35.946785
...,...,...
224,09_112,44.718544
225,09_113,44.718544
226,09_114,44.718544
227,09_115,44.718544


In [ ]:
submission.to_csv('v.0.0.2.csv',index=False)